The code below should let us vizualise and see the matches.

In [ ]:
import imageio
import numpy as np

def record_match(env, red_model, blue_model, filename="match_replay.gif"):
    """
    Runs a match between two agents and saves a GIF replay.
    """
    obs_red = env.get_obs_for_agent(1)
    # Note: We don't strictly need obs_blue for reset, but good to have
    obs_blue = env.get_obs_for_agent(2) 
    
    frames = []
    
    terminated = False
    truncated = False
    step = 0
    
    print(f"Recording match to {filename}...")
    
    while not (terminated or truncated):
        # 1. Capture the current frame
        frame = env.render()
        frames.append(frame)
        
        # 2. Get Actions from both Brains
        # We need to extract the 'image' part for the CNN
        action_red, _ = red_model.predict(obs_red['image'], deterministic=True)
        action_blue, _ = blue_model.predict(obs_blue['image'], deterministic=True)
        
        # 3. Step the environment
        obs_red, rew_red, obs_blue, rew_blue, done = env.step_multi_agent(action_red, action_blue)
        
        # Check termination logic from your env
        # Your step_multi_agent returns 'done' as the 5th element
        if done:
            terminated = True
        
        step += 1
        if step >= env.max_steps:
            truncated = True

    # Save the last winning frame
    frames.append(env.render())
    
    # Save to file
    imageio.mimsave(filename, frames, fps=10) # 10 frames per second
    print(f"Replay saved! ({len(frames)} frames)")